# 7 Applications to neuroscience

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib as mpl
import matplotlib.animation
import matplotlib.pyplot as plt
from scipy import io as sio
import pygsp as pg
import IPython.display as ipd

## 7.1 Brain data

The data is one subject from the [human connectome project dataset](https://db.humanconnectome.org/).
There are both a graph and graph signals:
* `SC.mat`: brain structural connectivity matrix obtained from MRI diffusion imaging (our graph),
* `RS_TCs.mat`: functional signals obtained from resting-state fMRI (our graph signals),
* `Glasser360_2mm_codebook.mat`: 3D positions of the nodes in MNI coordinates (to layout the graph).

For both modalities, the brain atlas from Glasser et al (Nature 2016) is used, parcellating the brain into 360 regions.

In [ ]:
datadir = Path('../data')

### 7.1.1 Structural connectivity graph

Load the connectivity matrix.

In [ ]:
connectivity = sio.loadmat(datadir / 'SC.mat')['SC']
n_regions, n_regions = connectivity.shape
print(f'{n_regions} brain regions')

Look at the structure of the connectivity matrix. Can you identify regions / communities?

In [ ]:
#plt.spy(connectivity);
plt.imshow(np.log(connectivity, out=np.zeros_like(connectivity), where=(connectivity!=0)));
plt.colorbar();

Look at the distribution of connection weights.

In [ ]:
plt.hist(connectivity.flatten(), bins=100, log=True);

Build a PyGSP graph from the connectivity matrix. Use the normalized Laplacian.

In [ ]:
graph = pg.graphs.Graph(connectivity, gtype='human connectome', lap_type='normalized')

print(f'{graph.N:_} nodes')
print(f'{graph.Ne:_} edges')
print(f'connected: {graph.is_connected()}')
print(f'directed: {graph.is_directed()}')

Look at the degree distribution.

In [ ]:
plt.hist(graph.dw, bins=100);

Look at the eigenvalues. If the graph is correct, you should find back the figure Giulia presented.

In [ ]:
graph.compute_fourier_basis()
plt.plot(graph.e);

Look at the graph, with a force-directed layout.

In [ ]:
graph.set_coordinates(kind='spring')
graph.plot()

Now embed the graph in 3D using the real positions of the regions. Look at it again.

In [ ]:
coordinates = sio.loadmat(datadir / 'Glasser360_2mm_codebook.mat')['codeBook']
graph.coords = np.stack([coordinates[0, 0][-2][0][region][:, 0] for region in range(n_regions)])

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
graph.plot(ax=ax)
ax.set_aspect('equal', adjustable='box')

Plot some of the first eigenvectors of the normalized Laplacian, on the 3D embedded graph. Can you identify how the brain is splitted?

In [ ]:
n_eigenvectors = 5

fig = plt.figure(figsize=(20, 5))
for i in range(n_eigenvectors):
    ax = fig.add_subplot(1, n_eigenvectors, i+1, projection='3d')
    graph.plot_signal(graph.U[:, i], ax=ax, colorbar=False)
    ax.set_title(f'eigenvector {i}')
    ax.axis('off')

Project the graph in 2D, by ignoring the z coordinate. Plot again some eigenvectors.

In [ ]:
graph.coords = graph.coords[:, :2]

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for i, ax in enumerate(axes):
    graph.plot_signal(graph.U[:, i], ax=ax, colorbar=True)
    ax.set_title(f'eigenvector {i}')
    ax.axis('off')

### 7.1.2 Functional activity (fMRI)

Load the fMRI signals.

In [ ]:
signals = sio.loadmat(datadir / 'RS_TCs.mat')['RS_TCs']
assert signals.shape[0] == n_regions

See how the signals are correlated across regions.

In [ ]:
plt.plot(signals.T);

In [ ]:
plt.plot(signals.mean(axis=0));

Look at some time slices of the fMRI signal on the functional connectivity graph.

In [ ]:
START = 200
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
for i, ax in enumerate(axes):
    graph.plot_signal(signals[:, START+i], ax=ax, colorbar=False)
    ax.set_title(f'time {START+i}')
    ax.axis('off')

Let's animate the functional activity on the structural graph.

In [ ]:
plt.rc('animation', embed_limit=100)
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect('equal', adjustable='box')
ax.axis('off')
plt.close(fig)
sc = ax.scatter(graph.coords[:, 0], graph.coords[:, 1])
cmap = plt.cm.get_cmap()

def animate(i):
    sc.set_color(cmap(signals[:, i]))
    ax.set_title(f'time {i}')
    return (sc,)

animation = mpl.animation.FuncAnimation(fig, animate, blit=True,
                                        frames=signals.shape[1], interval=20)

ipd.HTML(animation.to_jshtml())

Is the functional signal smooth on the structural graph? Does the smoothness (measured with the Dirichlet form) vary across time?

In [ ]:
smoothness = signals.T @ graph.L @ signals
plt.plot(np.diag(smoothness));

## 7.1 Analysis of fMRI signals on the brain connectome

In this exercise, you will:
1. look at the spectral content (w.r.t. the connectome) of functional signals,
1. decompose that signal in an aligned (low frequencies) and liberal (high frequencies) component,
1. identify which brain regions mostly support aligned and liberal activities. The norms of these reconstructed portions of the signal are considered as measures of alignment/liberality of the functional signal to the structure.

All the details and justifications are in the paper [Functional Alignment with Anatomical Networks is Associated with Cognitive Flexibility](https://arxiv.org/abs/1611.08751).

In [ ]:
# Number of eigenvalues to select for low frequencies (aligned functional activity).
THRESHOLD = 30 

graph.compute_fourier_basis()

Look at the norm (across time) of the spectral content of the fMRI signal.

In [ ]:
# Your code here.

Design a complementary pair of ideal low-pass and high-pass filters with a cutoff set at the eigenvalue `graph.e[THRESHOLD]`.

In [ ]:
# Your code here.

Decompose the fMRI signal in an aligned component (which corresponds to low frequencies) and a liberal component (which corresponds to high frequencies).

In [ ]:
# Your code here.

Plot the original functional signal along with its aligned and liberal decomposition. Plot as an image whose rows correspond to brain regions and columns to time.

In [ ]:
# Your code here.

Compute the norm (across time) of the aligned and liberal components. Plot those two norms as graph signals.

In [ ]:
# Your code here.

Can you identify the brain regions that correpond to activity aligned with the underlying structural graph from the regions that correspond to liberal activity? If you could reproduce the findings of the paper, sensory regions (visual, somatomotor, auditory) should correspond to aligned activity, and high-level cognition (multisensory integration, memory, decision making) should correpond to liberal activity.

## 7.2 Assessment of significant excursions in fMRI signals, wrt randomized timecourses

You can find all the details in the paper [A Spectral Method for Generating Surrogate Graph Signals](http://miplab.epfl.ch/pub/pirondini1601.pdf).

**TODO**: The graph randomization reported in Pirondini 2016 + test of the absolute value of the empirical fMRI signals versus the generated surrogates.